In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()







import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
episode = pd.read_csv('episode_data.csv',sep=';')
interviews = pd.read_csv('interviews.csv', sep=';')

In [3]:
episode_data = episode.copy()

In [4]:
def checkEpisode(date, patient):
    episode = 'N'
    ep = episode_data.loc[episode_data['patient'] == patient]
    for index, row in ep.iterrows():
        if date >= row.start and date < row.end:
            episode = row.episode
    return episode

In [5]:
interviews_episodes = interviews.copy()
for index, row in interviews_episodes.iterrows():
    interviews_episodes.at[index, 'episode'] = checkEpisode(row.date, row.patient)

In [6]:
interviews_episodes = interviews_episodes.drop('patient', 1)
interviews_episodes = interviews_episodes.drop('date', 1)

In [7]:
interviews_episodes = interviews_episodes.drop("alcohol", 1)
interviews_episodes = interviews_episodes.drop("other_drugs", 1)

In [8]:
interviews_episodes.drop(interviews_episodes.columns[interviews_episodes.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [9]:
interviews_episodes.head()

,mood,motivation,attention,irritability,anxiety,sleep_quality,nr_cigarettes,caffeine,active_time,episode
0,0,1,1,1,1,2,0,90,1470,N
1,-1,-1,2,1,1,3,24,120,1790,N
2,0,-1,2,1,1,1,24,90,1545,N
3,2,2,3,3,3,3,34,150,1710,N
4,2,2,3,3,3,3,38,150,1770,N


In [73]:
X_train, X_test, y_train, y_test = train_test_split(interviews_episodes.loc[:, interviews_episodes.columns != "episode"], 
                                                    interviews_episodes["episode"], test_size=0.3)


In [74]:
dt = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=15)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [75]:
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [76]:
svm = LinearSVC(dual=False)
svm.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [77]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
def check_state(classifier, data):
    prediction = classifier.predict(data)[0]
    if prediction == 'D':
        diagnosis = "The patient could be tending towards a Depression episode"
    elif prediction == 'M':
        diagnosis = "The patient could be tending towards a Mania episode"
    else:
        diagnosis = "The patient is in a Euthymic state"
    return diagnosis

In [79]:
from random import randint

In [80]:
n = 0
while n < 60:
    variables = {}
    variables['mood'] = randint(-3, 3)
    variables['motivation'] = randint(-3, 3)
    variables['attention'] = randint(0, 4)
    variables['irritability'] = randint(0, 4)
    variables['anxiety'] = randint(0, 4)
    variables['sleep_quality'] = randint(0, 4)
    variables['nr_cigarettes'] = randint(0, 40)
    variables['caffeine'] = randint(0, 250)
    variables['active_time'] = randint(200, 1800)
    
    test = np.array([variables['mood'], variables['motivation'], variables['attention'],
                variables['irritability'], variables['anxiety'], variables['sleep_quality'],
                variables['nr_cigarettes'], variables['caffeine'], variables['active_time']])
    
    
    
    print("--------------------------------------------")
    print("PATIENT ",n + 1)
    print("--------------------------------------------")
    print("Mood: ", variables['mood'])
    print("Motivation: ", variables['motivation'])
    print("Attention: ", variables['attention'])
    print("Irritability: ", variables['irritability'])
    print("Anxiety: ", variables['anxiety'])
    print("Sleep quality: ", variables['sleep_quality'])
    print("Number of cigarettes: ", variables['nr_cigarettes'])
    print("Caffeine: ", variables['caffeine'])
    print("Active time: ", variables['active_time'])
    
    
    test = test.reshape(1,-1)
    print("--------------------------------------------")
    print("PREDICTIONS")
    print("--------------------------------------------")
    print("- Decision Tree prediction: ", check_state(dt, test))
    print("- Random Forest prediction: ", check_state(rf, test))
    print("- Logistic Regression prediction: ", check_state(lr, test))
    test = scaler.fit_transform(test)
    print("- SVM prediction: ", check_state(svm, test))
    print("\n")
    
    
    n += 1

--------------------------------------------
PATIENT  1
--------------------------------------------
Mood:  3
Motivation:  3
Attention:  0
Irritability:  3
Anxiety:  3
Sleep quality:  2
Number of cigarettes:  2
Caffeine:  114
Active time:  851
--------------------------------------------
PREDICTIONS
--------------------------------------------
- Decision Tree prediction:  The patient is in a Euthymic state
- Random Forest prediction:  The patient is in a Euthymic state
- Logistic Regression prediction:  The patient is in a Euthymic state
- SVM prediction:  The patient is in a Euthymic state


--------------------------------------------
PATIENT  2
--------------------------------------------
Mood:  3
Motivation:  3
Attention:  0
Irritability:  2
Anxiety:  3
Sleep quality:  3
Number of cigarettes:  24
Caffeine:  76
Active time:  1041
--------------------------------------------
PREDICTIONS
--------------------------------------------
- Decision Tree prediction:  The patient is in a Euth